# Advanced Tuning — Metrics, Write Distribution, Bloom Filters

이 노트북에서는 Iceberg의 **세밀한 성능 튜닝** 옵션을 실습합니다.

기본적인 Compaction과 Partitioning으로 충분하지 않을 때, 메트릭 수집 레벨, 쓰기 분배 모드, Bloom Filter 등을 조정하여 추가 성능 개선을 달성할 수 있습니다.

## 환경 설정

In [ ]:
import sys
sys.path.append('..')

import time
import json
import glob as glob_mod
from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, count_files, total_size

In [ ]:
spark = create_spark_session()

METRICS_TABLE = "demo.lab.tuning_metrics"
METRICS_PATH = "/home/jovyan/data/warehouse/lab/tuning_metrics"

DIST_TABLE = "demo.lab.tuning_distribution"
DIST_PATH = "/home/jovyan/data/warehouse/lab/tuning_distribution"

BLOOM_TABLE = "demo.lab.tuning_bloom"
BLOOM_PATH = "/home/jovyan/data/warehouse/lab/tuning_bloom"

---
## 실험 1: Metrics Collection — 컬럼별 통계 수집 레벨

Iceberg는 각 데이터 파일에 대해 **컬럼별 통계**를 manifest 파일에 기록합니다. 이 통계는 쿼리 시 파일을 건너뛸 수 있게 해주는 핵심 정보입니다.

### 4가지 메트릭 모드

| 모드 | 수집 정보 | 용도 |
|------|----------|------|
| `none` | 통계 없음 | 넓은 테이블에서 메타데이터 크기 절약 |
| `counts` | null 수, 값 수 | 기본 정보만 필요할 때 |
| `truncate(N)` | counts + min/max (N바이트로 잘림) | **기본값** (N=16), 대부분의 경우 충분 |
| `full` | counts + 전체 min/max | 가장 정밀, 메타데이터 크기 증가 |

In [ ]:
spark.sql(f"DROP TABLE IF EXISTS {METRICS_TABLE}")

spark.sql(f"""
CREATE TABLE {METRICS_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

# 데이터 삽입
orders = generate_orders(num_records=500, seed=42)
df = to_spark_df(spark, orders)
df.writeTo(METRICS_TABLE).append()

print(f"테이블 생성 + 500건 삽입 완료: {METRICS_TABLE}")

In [ ]:
# 기본 메트릭 (truncate(16)) 확인
print("=== 기본 메트릭 (truncate(16)) — Manifest 파일 정보 ===")
spark.sql(f"""
SELECT 
    regexp_replace(file_path, '^file:.*?/(data/)', '$1') as file_path,
    record_count,
    lower_bounds,
    upper_bounds,
    null_value_counts,
    value_counts
FROM {METRICS_TABLE}.files
""").show(truncate=False)

In [ ]:
# 특정 컬럼의 메트릭 모드 변경: status는 필터링에 안 쓰므로 none으로
spark.sql(f"""
ALTER TABLE {METRICS_TABLE}
SET TBLPROPERTIES (
    'write.metadata.metrics.column.status' = 'none'
)
""")

print("status 컬럼 메트릭을 'none'으로 변경")
print("(새로 쓰는 파일부터 적용됨)")

In [ ]:
# 메트릭 변경 후 새 데이터 삽입
orders_new = generate_orders(num_records=100, seed=99, id_offset=501)
df_new = to_spark_df(spark, orders_new)
df_new.writeTo(METRICS_TABLE).append()

# 새 파일의 메트릭 확인
print("=== 메트릭 변경 후 새 파일 정보 ===")
spark.sql(f"""
SELECT 
    regexp_replace(file_path, '^file:.*?/(data/)', '$1') as file_path,
    record_count,
    lower_bounds,
    upper_bounds
FROM {METRICS_TABLE}.files
ORDER BY file_path DESC
""").show(truncate=False)

### 관찰 포인트 — Metrics Collection

- 기본적으로 모든 컬럼에 `truncate(16)` 통계가 수집됩니다
- **넓은 테이블**(컬럼 100개 이상)에서는 모든 컬럼의 통계를 수집하면 메타데이터가 매우 커집니다
- 필터링에 사용하지 않는 컬럼은 `none`으로 설정하여 메타데이터 크기를 줄일 수 있습니다
- 자주 필터링하는 핵심 컬럼은 `full`로 설정하여 프루닝 정밀도를 높일 수 있습니다

```sql
-- 넓은 테이블 전략 예시
ALTER TABLE my_table SET TBLPROPERTIES (
    'write.metadata.metrics.default' = 'none',              -- 기본: 통계 없음
    'write.metadata.metrics.column.user_id' = 'full',       -- 핵심 필터 컬럼만 full
    'write.metadata.metrics.column.created_at' = 'truncate(16)'
);
```

---
## 실험 2: Write Distribution Mode — 쓰기 분배 전략

Write Distribution Mode는 Spark가 데이터를 파일로 쓸 때 **레코드를 어떻게 분배할지** 결정합니다.

### 3가지 분배 모드

| 모드 | 동작 | 장점 | 단점 |
|------|------|------|------|
| `none` | 분배 없음, 각 태스크가 자기 데이터를 바로 작성 | 쓰기 속도 최고 | 파티션당 파일 폭증 가능 |
| `hash` | 파티션 키 기준 해시 셔플 | 파티션당 파일 수 제어 | 셔플 비용 발생 |
| `range` | 파티션 키 + sort 키 기준 범위 셔플 | 파일 내 정렬 보장 | 셔플 비용 가장 높음 |

In [ ]:
# none 모드 테이블
spark.sql(f"DROP TABLE IF EXISTS {DIST_TABLE}_none")
spark.sql(f"""
CREATE TABLE {DIST_TABLE}_none (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES ('write.distribution-mode' = 'none')
""")

# hash 모드 테이블
spark.sql(f"DROP TABLE IF EXISTS {DIST_TABLE}_hash")
spark.sql(f"""
CREATE TABLE {DIST_TABLE}_hash (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES ('write.distribution-mode' = 'hash')
""")

print("none / hash 모드 테이블 생성 완료")

In [ ]:
# 동일 데이터를 양쪽에 삽입하고 파일 구조 비교
orders = generate_orders(num_records=500, seed=42)
df = to_spark_df(spark, orders)

# none 모드
start = time.time()
df.writeTo(f"{DIST_TABLE}_none").append()
none_time = time.time() - start
none_files = count_files(f"{DIST_PATH}_none")

# hash 모드
start = time.time()
df.writeTo(f"{DIST_TABLE}_hash").append()
hash_time = time.time() - start
hash_files = count_files(f"{DIST_PATH}_hash")

print(f"{'모드':<8} {'쓰기 시간':>10} {'파일 수':>8}")
print("=" * 30)
print(f"{'none':<8} {none_time:>9.3f}s {none_files:>7}개")
print(f"{'hash':<8} {hash_time:>9.3f}s {hash_files:>7}개")

In [ ]:
# 파일 구조 시각적 비교
print("=== none 모드 파일 구조 ===")
show_tree(f"{DIST_PATH}_none", max_depth=2)

print("\n=== hash 모드 파일 구조 ===")
show_tree(f"{DIST_PATH}_hash", max_depth=2)

### 관찰 포인트 — Write Distribution Mode

- **none**: 셔플 없이 바로 작성하므로 빠르지만, Spark 태스크 수만큼 파일이 생길 수 있습니다
- **hash**: 파티션 키 기준으로 셔플하여 같은 파티션의 데이터를 모은 후 작성하므로 파일 수가 줄어듭니다
- **range**: hash에 더해 정렬까지 보장하지만 셔플 비용이 가장 높습니다

### 선택 가이드

| 상황 | 권장 모드 |
|------|----------|
| 스트리밍 수집 (SLA 빡빡) | `none` + 주기적 Compaction |
| 배치 적재 (파일 수 관리) | `hash` |
| 배치 적재 + 정렬 필요 | `range` |

---
## 실험 3: Bloom Filters — Point Lookup 성능 개선

### Bloom Filter 개념

Bloom Filter는 **"이 파일에 찾는 값이 있을 수 있는가?"**를 빠르게 판단하는 확률적 자료구조입니다.

```
Bloom Filter 응답:
  "없다" → 확실히 없음 (True Negative) → 파일 스킵!
  "있다" → 있을 수도 있음 (True/False Positive) → 파일 읽기
```

### 동작 원리

1. 비트 배열(예: 1024비트)을 0으로 초기화
2. 각 값을 K개의 해시 함수로 해시하여 비트 위치를 계산
3. 해당 위치의 비트를 1로 설정
4. 검색 시: 값을 해시하여 모든 비트가 1인지 확인

### 적합한 경우

- `WHERE user_id = 'abc123'` 같은 **Point Lookup** 쿼리가 빈번할 때
- **고카디널리티 컬럼** (값의 종류가 매우 많은 컬럼)에 효과적

In [ ]:
spark.sql(f"DROP TABLE IF EXISTS {BLOOM_TABLE}")

# Bloom Filter 없는 기본 테이블
spark.sql(f"""
CREATE TABLE {BLOOM_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

# 대량 데이터 삽입 (여러 배치로 파일 분산)
for i in range(5):
    orders = generate_orders(num_records=200, seed=i, id_offset=i*200+1)
    df = to_spark_df(spark, orders)
    df.writeTo(BLOOM_TABLE).append()

total = spark.sql(f"SELECT COUNT(*) FROM {BLOOM_TABLE}").collect()[0][0]
print(f"테이블 생성 + {total}건 삽입 완료")
print(f"파일 수: {count_files(BLOOM_PATH)}")

In [ ]:
# Bloom Filter 없이 Point Lookup 성능 측정
times_without_bloom = []
for _ in range(3):
    start = time.time()
    spark.sql(f"SELECT * FROM {BLOOM_TABLE} WHERE order_id = 42").collect()
    times_without_bloom.append(time.time() - start)

avg_without = sum(times_without_bloom) / len(times_without_bloom)
print(f"Bloom Filter 없이 Point Lookup 평균: {avg_without:.3f}초")

In [ ]:
# Bloom Filter 활성화
spark.sql(f"""
ALTER TABLE {BLOOM_TABLE}
SET TBLPROPERTIES (
    'write.parquet.bloom-filter-enabled.column.order_id' = 'true',
    'write.parquet.bloom-filter-max-bytes' = '1048576'
)
""")

print("order_id 컬럼에 Bloom Filter 활성화")
print("(기존 파일에는 적용되지 않음 — Compaction이나 새 쓰기 시 적용)")

In [ ]:
# Compaction으로 Bloom Filter가 포함된 새 파일 생성
spark.sql(f"""
CALL demo.system.rewrite_data_files(
    table => '{BLOOM_TABLE}',
    strategy => 'binpack'
)
""").show(truncate=False)

print(f"Compaction 후 파일 수: {count_files(BLOOM_PATH)}")

In [ ]:
# Bloom Filter 적용 후 Point Lookup 성능 측정
times_with_bloom = []
for _ in range(3):
    start = time.time()
    spark.sql(f"SELECT * FROM {BLOOM_TABLE} WHERE order_id = 42").collect()
    times_with_bloom.append(time.time() - start)

avg_with = sum(times_with_bloom) / len(times_with_bloom)
print(f"Bloom Filter 없이: {avg_without:.3f}초")
print(f"Bloom Filter 있음: {avg_with:.3f}초")

if avg_without > 0:
    improvement = (avg_without - avg_with) / avg_without * 100
    print(f"개선율: {improvement:.1f}%")
    if improvement < 0:
        print("(작은 데이터셋에서는 Bloom Filter 오버헤드로 오히려 느릴 수 있음)")

### 관찰 포인트 — Bloom Filters

- Bloom Filter는 **Point Lookup** (`WHERE col = value`) 쿼리에서 불필요한 파일을 빠르게 건너뛸 수 있게 합니다
- 소규모 데이터셋에서는 효과가 미미하거나 오히려 오버헤드가 될 수 있습니다
- **대규모 데이터 + 고카디널리티 컬럼 + Point Lookup**이 빈번한 환경에서 가장 효과적입니다
- Bloom Filter는 **Parquet 파일 레벨**에 저장되므로, 활성화 후 **Compaction이나 새 쓰기가 필요**합니다

> 주의: Bloom Filter는 파일 크기를 약간 증가시킵니다. `bloom-filter-max-bytes`로 크기를 제한하세요.

---
## Object Storage 최적화

S3 같은 오브젝트 스토리지에서는 **prefix throttling** 문제가 발생할 수 있습니다.

### 문제

S3는 prefix(경로 접두사)별로 요청 속도를 제한합니다. 같은 파티션 디렉토리에 대량의 파일이 쓰이면 스로틀링이 발생할 수 있습니다.

### 해결: Object Storage Path

```sql
ALTER TABLE my_table SET TBLPROPERTIES (
    'write.object-storage.enabled' = 'true',
    'write.data.path' = 's3://bucket/data'
);
```

활성화하면 Iceberg가 **파일 경로에 해시를 추가**하여 다양한 prefix에 분산 저장합니다:

```
# Before (기본)
s3://bucket/warehouse/my_table/data/order_date_month=2024-01/file1.parquet
s3://bucket/warehouse/my_table/data/order_date_month=2024-01/file2.parquet

# After (Object Storage 활성화)
s3://bucket/data/a1b2c3d4/order_date_month=2024-01/file1.parquet
s3://bucket/data/e5f6a7b8/order_date_month=2024-01/file2.parquet
```

파일 위치가 분산되지만, Iceberg 메타데이터가 파일 경로를 추적하므로 쿼리에는 영향이 없습니다.

---
## 튜닝 옵션 요약

| 옵션 | 기본값 | 튜닝 포인트 | 효과 |
|------|--------|------------|------|
| **Metrics** | `truncate(16)` | 넓은 테이블 → 핵심만 `full`, 나머지 `none` | 메타데이터 크기 절약 |
| **Distribution** | `none` | 배치 → `hash`, SLA 빡빡 → `none` | 파일 수 제어 |
| **Bloom Filter** | 비활성 | 고카디널리티 + Point Lookup → 활성화 | 파일 스킵 |
| **Object Storage** | 비활성 | S3 대량 쓰기 → 활성화 | prefix 스로틀링 방지 |

### 언제 튜닝이 필요한가?

1. **Compaction + Partitioning만으로 충분하지 않을 때**
2. **넓은 테이블**(100+ 컬럼)에서 메타데이터 크기가 문제일 때 → Metrics 조정
3. **스트리밍 수집 후 파일이 너무 많을 때** → Distribution Mode 조정
4. **특정 값 검색이 느릴 때** → Bloom Filter 활성화
5. **S3 쓰기 성능이 떨어질 때** → Object Storage 경로 활성화

> 대부분의 경우 Compaction + Partitioning이면 충분합니다. 이 튜닝들은 **문제가 확인된 후** 적용하세요.

In [ ]:
spark.stop()
print("Spark 세션 종료")